# Dlib vs MediaPipe Face Mesh Comparison

#### Enviroment preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

%cd /content

!rm -rf automatic-pain-recognition

print(f"Obecne położenie: {os.getcwd()}")

/content
Obecne położenie: /content


In [ ]:
USERNAME = 'alicka33'
REPO_NAME = 'automatic-pain-recognition'

REPO_URL = f"https://github.com/{USERNAME}/{REPO_NAME}.git"

print("Rozpoczynam klonowanie...")

!git clone {REPO_URL}

%cd $REPO_NAME

In [ ]:
!pip install -r requirements.txt

In [ ]:
import sys
import os

sys.path.append(os.getcwd())
print(f"Dodano do ścieżki Pythona: {os.getcwd()}")

Dodano do ścieżki Pythona: /content/automatic-pain-recognition


#### MediaPipe Compatibility Fix (Python 3.12+ / January 2026)
Note: Since the Google Colab update in early 2026, MediaPipe has changed its package structure. If you encounter AttributeError: module 'mediapipe' has no attribute 'solutions' or ModuleNotFoundError, please run the two cells below.

In [ ]:
!pip uninstall -y mediapipe
!rm -rf /usr/local/lib/python3.12/dist-packages/mediapipe

!pip install --no-cache-dir mediapipe==0.10.14

import os
os.kill(os.getpid(), 9)

A comparison between the Dlib and MediaPipe Face Mesh in order to determine which extraction method is more efficient for training and real time pain detection

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from IPython.display import display

from data_preparation.processing_pipeline_dlib import init_dlib, video_to_landmark_vectors as dlib_video_to_vectors
from data_preparation.processing_pipeline_mediapipe import load_reference_keypoints, video_to_feature_sequences as mp_video_to_feature_sequences

print("Imports ready.")

Imports ready.


In [ ]:
COLAB_ROOT = '/content/drive/MyDrive/PainRecognitionProject/'
DATA_DIR = os.path.join(COLAB_ROOT, 'data/BioVid_HeatPain/')
FRAME_SKIP_VALUE = 3

PREDICTOR_PATH = os.path.join(COLAB_ROOT, 'data', 'shape_predictor_68_face_landmarks.dat')
MEAN_FACE_PATH = os.path.join(COLAB_ROOT, 'data', 'landmarks_mean_face.npy')
WEIGHTS_PATH = os.path.join(COLAB_ROOT, 'data', 'frontalization_weights.npy')

def list_sample_videos(base_dir, limit=20):
    samples = []
    for subj in sorted(os.listdir(base_dir)):
        subj_path = os.path.join(base_dir, subj)
        if not os.path.isdir(subj_path):
            continue
        for fn in sorted(os.listdir(subj_path)):
            if fn.lower().endswith('.mp4'):
                samples.append(os.path.join(subj_path, fn))
                if len(samples) >= limit:
                    return samples
    return samples

sample_videos = list_sample_videos(DATA_DIR, limit=50)
print("Found sample videos:", len(sample_videos))
VIDEO_SAMPLE_PATH = sample_videos[0] if sample_videos else None
print("Default sample:", VIDEO_SAMPLE_PATH)

Found sample videos: 50
Default sample: /content/drive/MyDrive/PainRecognitionProject/data/BioVid_HeatPain/071309_w_21/071309_w_21-BL1-081.mp4


In [7]:
dlib_ctx = init_dlib(PREDICTOR_PATH, MEAN_FACE_PATH, WEIGHTS_PATH)
print("Dlib initialized. Frontalization weights loaded:", dlib_ctx.get('frontalization_weights') is not None)

mp_ref_kp, mp_ref_ok = load_reference_keypoints(os.path.join(COLAB_ROOT, 'data', 'key_points_xyz.npy'))
print("MediaPipe reference loaded:", mp_ref_ok)

Dlib initialized. Frontalization weights loaded: True
MediaPipe reference loaded: True


In [8]:
def dlib_extractor_wrapper(video_path, frame_skip, visualize=False):
    return dlib_video_to_vectors(
        video_path=video_path,
        detector=dlib_ctx['detector'],
        predictor=dlib_ctx['predictor'],
        aligner=dlib_ctx.get('aligner'),
        frontalization_weights=dlib_ctx.get('frontalization_weights'),
        canonical_reference=dlib_ctx.get('canonical_reference'),
        frame_skip=frame_skip,
        frontalize=True,
        visualize=visualize
    )

def mp_extractor_wrapper(video_path, frame_skip, visualize=False):
    return mp_video_to_feature_sequences(
        video_path=video_path,
        frame_skip=frame_skip,
        reference_keypoints_3d=mp_ref_kp if mp_ref_ok else None,
        use_frontalization=bool(mp_ref_ok),
        visualize=visualize
    )

In [9]:
def run_comparison(method_name, extraction_function, video_path, skip):
    if extraction_function is None:
        raise RuntimeError(f"Extractor for {method_name} is not available.")
    print(f"\n--- Testing: {method_name} (frame_skip={skip}) ---")
    start = time.time()
    seq = extraction_function(video_path=video_path, frame_skip=skip, visualize=False)
    elapsed = time.time() - start

    if not seq:
        n_frames = 0; vector_len = 0; vector_kb = 0.0
        print("No feature vectors extracted.")
    else:
        arr = np.array(seq)
        n_frames = arr.shape[0]
        vector_len = arr.shape[1] if arr.ndim >= 2 else (arr[0].shape[0] if n_frames>0 else 0)
        vector_kb = (arr[0].nbytes if n_frames>0 else 0) / 1024.0
        print(f" - Shape (N,features): {arr.shape}")

    fps = n_frames / elapsed if elapsed > 0 else 0.0
    print(f" - Time: {elapsed:.2f}s | FPS: {fps:.2f} | Frames: {n_frames}")
    print(f" - Vector length: {vector_len} | Vector size: {vector_kb:.3f} KB")

    return {'method': method_name, 'time_s': elapsed, 'fps': fps, 'n_frames': n_frames, 'vector_len': vector_len, 'vector_kb': vector_kb}

In [ ]:
if VIDEO_SAMPLE_PATH is None:
    raise FileNotFoundError("No sample video found. Update VIDEO_SAMPLE_PATH or check DATA_DIR.")

results = []
results.append(run_comparison("Dlib (68 pts, 2D + Frontalization)", dlib_extractor_wrapper, VIDEO_SAMPLE_PATH, FRAME_SKIP_VALUE))
results.append(run_comparison("MediaPipe (478 pts, 3D + Frontalization)", mp_extractor_wrapper, VIDEO_SAMPLE_PATH, FRAME_SKIP_VALUE))

df_results = pd.DataFrame(results).set_index('method')
display(df_results.sort_values(by='fps', ascending=False))


--- Testing: Dlib (68 pts, 2D + Frontalization) (frame_skip=3) ---
 - Shape (N,features): (46, 136)
 - Time: 36.32s | FPS: 1.27 | Frames: 46
 - Vector length: 136 | Vector size: 0.531 KB

--- Testing: MediaPipe (478 pts, 3D + Frontalization) (frame_skip=3) ---


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


 - Shape (N,features): (46, 1434)
 - Time: 1.79s | FPS: 25.63 | Frames: 46
 - Vector length: 1434 | Vector size: 5.602 KB


,time_s,fps,n_frames,vector_len,vector_kb
method,,,,,
"MediaPipe (478 pts, 3D + Frontalization)",1.794593,25.632548,46,1434,5.601562
"Dlib (68 pts, 2D + Frontalization)",36.323171,1.266409,46,136,0.531250


In [ ]:
SAMPLES_TO_TEST = sample_videos[:10]
REPEAT_PER_SAMPLE = 1
aggregate = []

for vid in SAMPLES_TO_TEST:
    for _ in range(REPEAT_PER_SAMPLE):
        try:
            aggregate.append(run_comparison("Dlib (68 pts, 2D + Frontalization)", dlib_extractor_wrapper, vid, FRAME_SKIP_VALUE))
            aggregate.append(run_comparison("MediaPipe (478 pts, 3D + Frontalization)", mp_extractor_wrapper, vid, FRAME_SKIP_VALUE))
        except Exception as e:
            print("Run error:", e)

if aggregate:
    df = pd.DataFrame(aggregate)
    summary = df.groupby('method').agg({
        'time_s': ['mean','std'],
        'fps': ['mean','std'],
        'n_frames': 'mean',
        'vector_len': 'mean',
        'vector_kb': ['mean','std']
    })
    display(summary)
else:
    print("No data collected.")


--- Testing: Dlib (68 pts, 2D + Frontalization) (frame_skip=3) ---
 - Shape (N,features): (46, 136)
 - Time: 37.44s | FPS: 1.23 | Frames: 46
 - Vector length: 136 | Vector size: 0.531 KB

--- Testing: MediaPipe (478 pts, 3D + Frontalization) (frame_skip=3) ---


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


 - Shape (N,features): (46, 1434)
 - Time: 2.42s | FPS: 19.02 | Frames: 46
 - Vector length: 1434 | Vector size: 5.602 KB

--- Testing: Dlib (68 pts, 2D + Frontalization) (frame_skip=3) ---
 - Shape (N,features): (46, 136)
 - Time: 39.49s | FPS: 1.16 | Frames: 46
 - Vector length: 136 | Vector size: 0.531 KB

--- Testing: MediaPipe (478 pts, 3D + Frontalization) (frame_skip=3) ---
 - Shape (N,features): (46, 1434)
 - Time: 1.60s | FPS: 28.77 | Frames: 46
 - Vector length: 1434 | Vector size: 5.602 KB

--- Testing: Dlib (68 pts, 2D + Frontalization) (frame_skip=3) ---
 - Shape (N,features): (46, 136)
 - Time: 37.93s | FPS: 1.21 | Frames: 46
 - Vector length: 136 | Vector size: 0.531 KB

--- Testing: MediaPipe (478 pts, 3D + Frontalization) (frame_skip=3) ---
 - Shape (N,features): (46, 1434)
 - Time: 1.63s | FPS: 28.30 | Frames: 46
 - Vector length: 1434 | Vector size: 5.602 KB

--- Testing: Dlib (68 pts, 2D + Frontalization) (frame_skip=3) ---
 - Shape (N,features): (46, 136)
 - Time: 

time_s                  fps  \
                                               mean       std       mean   
method                                                                     
Dlib (68 pts, 2D + Frontalization)        37.392962  1.277123   1.231457   
MediaPipe (478 pts, 3D + Frontalization)   1.887088  0.364102  25.117708   

                                                   n_frames vector_len  \
                                               std     mean       mean   
method                                                                   
Dlib (68 pts, 2D + Frontalization)        0.041616     46.0      136.0   
MediaPipe (478 pts, 3D + Frontalization)  4.285651     46.0     1434.0   

                                         vector_kb       
                                              mean  std  
method                                                   
Dlib (68 pts, 2D + Frontalization)        0.531250  0.0  
MediaPipe (478 pts, 3D + Frontalization)  5.601562  0.0